<a href="https://colab.research.google.com/github/ilham-mukti/Machine-Learning/blob/main/data_teks/Document_Similarity_Data_Tirto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

import numpy as np
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

sw_indo = stopwords.words("indonesian") + list(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sw_indo

In [6]:
datas = pd.read_csv("/content/data_tirto.csv")
datas.head()

,id_berita,judul_berita,isi_berita,nama_penulis,date_created,url_artikel
0,1259889,Jokowi Sindir ASN yang Minta Dilayani: Seperti...,Presiden Jokowi menilai para aparatur sipil ne...,Andrian Pratama Taher,27/07/2021 10:55,https://tirto.id/jokowi-sindir-asn-yang-minta-...
1,1259829,Larangan Demo Jokowi End Game & Bahaya Doxing ...,"Jumat, 23 Juli 2021, sekira pukul 23-an, ada t...",Adi Briantika,26/07/2021 17:24,https://tirto.id/larangan-demo-jokowi-end-game...
2,1259921,"Jokowi: Mahasiswa Punya Talenta, Bisa Berprofe...",Presiden Joko Widodo meminta agar perguruan ti...,Andrian Pratama Taher,27/07/2021 13:36,https://tirto.id/jokowi-mahasiswa-punya-talent...
3,1259902,Tjahjo Minta Seluruh ASN Ikuti Arahan Jokowi-M...,Menteri Pendayagunaan Aparatur Negara dan Refo...,Andrian Pratama Taher,27/07/2021 12:30,https://tirto.id/tjahjo-minta-seluruh-asn-ikut...
4,1259792,"Jokowi, Menkes dan Mendag Disomasi Imbas Kelan...",Koalisi masyarakat sipil menyomasi Presiden Jo...,Irwan Syambudi,26/07/2021 13:41,https://tirto.id/jokowi-menkes-dan-mendag-diso...


In [7]:
datas.drop_duplicates(subset='id_berita', inplace=True, ignore_index=True)

In [8]:
datas.shape

(863, 6)

# Ekstrak TFIDF

In [9]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(datas.isi_berita)
tfidf_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<863x243099 sparse matrix of type '<class 'numpy.float64'>'
	with 475069 stored elements in Compressed Sparse Row format>

# Document Similarity

In [26]:
#Kalo pake Distance
def document_similarity(index, matrix, topk=6):
  sim = cosine_distances(matrix[index], matrix)
  sorted_sim = sim.argsort()[0, 0:topk+1]
  return sorted_sim

In [27]:
data_ke = 0
result_index = document_similarity(data_ke, tfidf_matrix)
print(result_index)

[  0   3 616  82 108 140   2]


In [12]:
datas.isi_berita[0][:400]

'Presiden Jokowi menilai para aparatur sipil negara (ASN) di masa depan harus mampu melayani masyarakat. Ia tidak mau ada ASN justru bermental seperti aparatur di masa kolonial yang ingin dilayani oleh masyarakat.\xa0"Setiap ASN harus mempunyai orientasi yang sama yaitu memberikan pelayanan terbaik untuk masyarakat. ASN bukan pejabat yang justru minta dilayani, yang bergaya seperti pejabat zaman kolon'

In [13]:
datas.isi_berita[3][:400]

'Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi (PAN-RB) Tjahjo Kumolo meminta kepada seluruh aparatur sipil negara (ASN) untuk bersatu padu dalam penanganan COVID-19. Ia meminta kepada semua pihak terutama para ASN untuk patuh arahan Jokowi."Kami mengajak seluruh ASN sebagai bagian dari pada aparatur instansi pemerintah, bersama-sama dengan elemen masyarakat, tokoh masyarakat, ormas'

In [14]:
datas.isi_berita[616][:400]

'Pemerintah Kabupaten Boyolali di Provinsi Jawa Tengah mulai Senin (14/9/2020) akan menerapkan kebijakan kerja dari rumah atau WFH bagi aparatur sipil negara (ASN) dalam upaya meminimalkan risiko penularan COVID-19 yang terus bertambah. Sekretaris Daerah Pemerintah Kabupaten Boyolali Masruri di Boyolali, Jumat (11/9/2020), menjelaskan sesuai dengan Surat Edaran Bupati mengenai sistem kerja ASN dala'

In [33]:
datas.isi_berita[2][:400]

'Presiden Joko Widodo meminta agar perguruan tinggi untuk mengubah dan memfasilitasi kebutuhan pembelajaran untuk kebutuhan masa depan. Jokowi menilai, para mahasiswa harus mendapatkan ilmu pengetahuan di tengah rangkaian distrupsi akibat revolusi industri 4.0 yang dipercepat dengan pandemi COVID-19. Pembelajaran jarak jauh di kampus sudah berlangsung di awal pandemi. Semua pembelajaran berlangsung'

In [16]:
datas.loc[result_index, 'id_berita': 'nama_penulis']

,id_berita,judul_berita,isi_berita,nama_penulis
0,1259889,Jokowi Sindir ASN yang Minta Dilayani: Seperti...,Presiden Jokowi menilai para aparatur sipil ne...,Andrian Pratama Taher
3,1259902,Tjahjo Minta Seluruh ASN Ikuti Arahan Jokowi-M...,Menteri Pendayagunaan Aparatur Negara dan Refo...,Andrian Pratama Taher
616,1211577,"Cegah Penularan COVID-19, Boyolali Terapkan Ke...",Pemerintah Kabupaten Boyolali di Provinsi Jawa...,tidak ada
82,1259654,"Soal Demo Jokowi End Game, Mahfud: Ada Kelompo...",Menkopolhukam Mahfud MD menyebut ada sekelompo...,Andrian Pratama Taher


In [17]:
#Kalo pake Similarity, hasilnya sama aja
def document_similarity(index, matrix, topk=3):
  sim = cosine_similarity(matrix[index], matrix)
  sorted_sim = sim[0].argsort()
  berita_yang_sama = [index for index in reversed(sorted_sim[-topk:])]
  return berita_yang_sama

In [18]:
data_ke = 1
result_index = document_similarity(data_ke, tfidf_matrix)
print(result_index)

[1, 87, 95]


In [19]:
datas.loc[result_index, 'id_berita': 'nama_penulis']

,id_berita,judul_berita,isi_berita,nama_penulis
1,1259829,Larangan Demo Jokowi End Game & Bahaya Doxing ...,"Jumat, 23 Juli 2021, sekira pukul 23-an, ada t...",Adi Briantika
87,1259634,Polisi Tangkap 2 Terduga Inisiator Demo Jokowi...,Polisi meringkus dua orang di Semarang yang me...,Andrian Pratama Taher
95,1259592,Polisi Khawatir Demo &#039;Jokowi End Game&#03...,Polri mengimbau masyarakat tidak terhasut deng...,Adi Briantika


In [20]:
def extract_keyword_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return([vocab[idx] for idx in reversed(sorted_tfidf[-topk:])])

In [21]:
text = datas.isi_berita[19]
extract_keyword_tfidf(text, tfidf)

['stigmatisasi',
 'isoman',
 'kematian',
 'covid',
 'vino',
 'budi',
 'sulfikar',
 'menkes budi',
 'penderita covid',
 'penderita']